In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

 Activating environment at `~/Research/altro-mpc-icra2021/benchmarks/random_linear_mpc/Project.toml`


# Generate Random Problem
TODO: include mathematical definition of MPC problem here.

In [2]:
include("random_linear.jl")

# using ControlSystems

n = 10
m = Int(n/2)
N = 20
A,B = gendiscrete(n,m)

q = 10*rand(n)
r = 0.1*ones(m)
Q = Diagonal(q)
R = Diagonal(r)

Qf = Q #dare(A, B, Q, R)

x̄ = rand(n) .+ 1
ū = 0.1*rand(m)
x0 = (rand(n) .- 1) .* x̄ * 0.5
x0_warm = (rand(n) .- 1) .* x̄ * 0.5
xf = zeros(n)

tol = 1e-4

nothing

# Solve Using Altro
First, we setup the random linear MPC problem and use benchmark_solve! to characterize the solve. 

In [3]:
using Altro
using TrajectoryOptimization
using RobotDynamics

const RD = RobotDynamics

dt = 0.1 # doesn't matter, just needs to be non-zero
model = RD.LinearModel(A, B; dt=dt)
objective = LQRObjective(Q, R, Qf, xf, N)

constraints = ConstraintList(n, m, N)
bound = BoundConstraint(n, m, x_min=-x̄, x_max=x̄, u_min=-ū, u_max=ū)
add_constraint!(constraints, bound, 1:N)

tf = (N-1)*dt

problem = Problem(model, objective, xf, tf, x0=x0, constraints=constraints, integration=RD.PassThrough)
solver = ALTROSolver(problem)
set_options!(solver, projected_newton=false)
solve!(solver)

X_altro = states(solver)
U_altro = controls(solver)

b = benchmark_solve!(solver)

@show max_violation(solver)
@show cost(solver)
b

max_violation(solver) = 1.2067504220935124e-7
cost(solver) = 1.0518199466952804


BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.120 ms (0.00% GC)
  median time:      2.303 ms (0.00% GC)
  mean time:        2.320 ms (0.00% GC)
  maximum time:     2.693 ms (0.00% GC)
  --------------
  samples:          10
  evals/sample:     10

Now we use the previous solution to warmstart the problem with a new initial state and benchmark the performance.

In [4]:
initial_controls!(solver, U_altro)
initial_states!(solver, X_altro)
set_options!(solver, reset_duals=false, penalty_initial=1.0)

# update x0 to see warmstarted solution:
solver.solver_al.solver_uncon.x0 .= x0_warm

b = benchmark_solve!(solver)

@assert states(solver)[1] ≈ x0_warm

b

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     246.985 μs (0.00% GC)
  median time:      277.262 μs (0.00% GC)
  mean time:        301.004 μs (0.00% GC)
  maximum time:     434.947 μs (0.00% GC)
  --------------
  samples:          10
  evals/sample:     10

# Solve Using OSQP

In [11]:
using LinearAlgebra
using JuMP
using OSQP
using BenchmarkTools

select(i, n) = (n*(i-1)+1):(n*(i-1)+n)

jump_model = Model(
    optimizer_with_attributes(
        OSQP.Optimizer, "eps_abs" => tol, "eps_rel" => tol, "eps_prim_inf" => tol, "eps_dual_inf" => tol 
    )
)
set_silent(jump_model)

Ī = zeros(n*(N-1), n)
Ā = zeros(n*(N-1), n)
B̄ = zeros(n*(N-1), m*(N-1))
for i=1:(N-1)
    Ā[select(i,n), :] = A^i
    Ī[select(i,n), :] = I(n)
    for j=1:i
        B̄[select(i, n), select(j, m)] = A^(i-1)*B
    end
end

Q̄ = Diagonal(repeat(q, N-1))
R̄ = Diagonal(repeat(r, N-1))

@variable(jump_model, u[1:((N-1)*m)])

objective_exp = @expression(jump_model, 2*transpose(Ā*x0)*Q̄*B̄*u + transpose(B̄*u)*Q̄*B̄*u + transpose(u)*R̄*u)

@constraint(jump_model, Ā*x0 + B̄*u .<= Ī*x̄)
@constraint(jump_model, Ā*x0 + B̄*u .>= -Ī*x̄)

for i=1:N-1
    # control/state bound constraints
    @constraint(jump_model, u[select(i, m)] .<= ū)
    @constraint(jump_model, u[select(i, m)] .>= -ū)
end

@objective(jump_model, Min, objective_exp)

optimize!(jump_model)

X̄ = Ā*x0 + B̄*value.(u)
X_osqp = [i==1 ? x0 : X̄[select(i-1,n)] for i=1:N]
U_osqp = [value.(u)[select(i, m)] for i=1:N-1]

@show objective_value(jump_model)

@benchmark optimize!($jump_model) samples=10 evals=10

objective_value(jump_model) = -9.275405223407585


BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.521 ms (0.00% GC)
  median time:      1.553 ms (0.00% GC)
  mean time:        1.661 ms (0.00% GC)
  maximum time:     2.031 ms (0.00% GC)
  --------------
  samples:          10
  evals/sample:     10

In [12]:
objective_value_(X, U) = sum([0.5*X[i]'*Q*X[i] for i=1:N]) + sum([0.5*U[i]'*R*U[i] for i=1:N-1])

@show objective_value_(X_altro, U_altro)
@show objective_value_(X_osqp, U_osqp)

objective_value_(X_altro, U_altro) = 10.517836387242044
objective_value_(X_osqp, U_osqp) = 11.227390983811679


11.227390983811679